https://github.com/JuliaStats/StatsModels.jl/issues/220

## Reproduce the slow-down (1)

In [1]:
using DataFrames
using GLM
using StatsBase: sample

n = 20
r = 10
x_symbols = [Symbol("x$i") for i in 1:n]
@time df = DataFrame(rand(100, n+1), [:y; x_symbols]);

  0.493259 seconds (1.89 M allocations: 108.855 MiB, 18.17% gc time, 99.95% compilation time)


In [2]:
x_vars = sample(x_symbols, r; replace=false)
@time F = term(:y) ~ sum(term(x) for x in x_vars)
@time cols = Tables.columntable(df)
@time mf = ModelFrame(F, cols, model=LinearModel)
@time mm = ModelMatrix(mf)
@time y = response(mf)
@time linmodel = fit(LinearModel, mm.m, y)
@time regmodel = StatsModels.TableRegressionModel(linmodel, mf, mm)

  0.147749 seconds (279.83 k allocations: 18.107 MiB, 41.16% gc time, 100.84% compilation time)
  0.230033 seconds (653.37 k allocations: 41.294 MiB, 3.72% gc time, 43.44% compilation time)
  1.582605 seconds (5.92 M allocations: 371.880 MiB, 4.99% gc time)
  0.308015 seconds (1.19 M allocations: 75.524 MiB, 5.53% gc time)
  0.005978 seconds (3.78 k allocations: 283.598 KiB, 99.78% compilation time)
  1.140388 seconds (5.13 M allocations: 290.797 MiB, 7.13% gc time, 0.12% compilation time)
  0.003558 seconds (3.76 k allocations: 276.488 KiB, 95.94% compilation time)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x5 + x11 + x15 + x17 + x19 + x1 + x16 + x20 + x10 + x8

Coefficients:
───────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   0.565795      0.192463   2.94    0.0042   0.183376   0.948215
x5            0.187196      0.109817   1.70    0.0918  -0.031009   0.4054
x11          -0.00207608    0.116319  -0.02    0.9858  -0.233199   0.229047
x15          -0.0702746     0.107201  -0.66    0.5138  -0.28328    0.142731
x17           0.0289332     0.105834   0.27    0.7852  -0.181357   0.239223
x19          -0.0515037     0.103151  -0.50    0.6188  -0.256463   0.153455
x1           -0.0993468     0.106629  -0.93    0.3540  -0.3112

In [3]:
x_vars = sample(x_symbols, r; replace=false)
@time F = term(:y) ~ sum(term(x) for x in x_vars)
@time cols = Tables.columntable(df)
@time mf = ModelFrame(F, cols, model=LinearModel)
@time mm = ModelMatrix(mf)
@time y = response(mf)
@time linmodel = fit(LinearModel, mm.m, y)
@time regmodel = StatsModels.TableRegressionModel(linmodel, mf, mm)

  0.097383 seconds (79.69 k allocations: 5.129 MiB, 12.89% gc time, 99.47% compilation time)
  0.000024 seconds (29 allocations: 1.766 KiB)
  0.055787 seconds (136.59 k allocations: 8.852 MiB, 7.31% compilation time)
  0.061263 seconds (80.33 k allocations: 5.021 MiB, 99.70% compilation time)
  0.006126 seconds (3.77 k allocations: 282.848 KiB, 99.77% compilation time)
  0.000026 seconds (21 allocations: 23.188 KiB)
  0.004769 seconds (2.75 k allocations: 194.801 KiB, 90.90% compilation time)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x9 + x12 + x5 + x10 + x15 + x20 + x8 + x19 + x2 + x16

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.510752    0.176476    2.89    0.0048   0.160097    0.861406
x9            0.22092     0.108066    2.04    0.0439   0.00619397  0.435645
x12          -0.0326253   0.113492   -0.29    0.7744  -0.258132    0.192882
x5            0.176219    0.107628    1.64    0.1051  -0.0376343   0.390073
x10          -0.0275409   0.104425   -0.26    0.7926  -0.235031    0.17995
x15          -0.0467616   0.103191   -0.45    0.6515  -0.251799    0.158276
x20          -0.0778703   0.102674   -0.76    0.4502  -0.28

## Reproduce the slow-down (2)

In [4]:
using DataFrames
using GLM
using StatsBase: sample

n = 20
r = 10
x_symbols = [Symbol("x$i") for i in 1:n]
df = DataFrame(rand(100, n+1), [:y; x_symbols]);

result = []
for _ in 1:10
    x_vars = sample(x_symbols, r; replace=false)
    F = term(:y) ~ sum(term(x) for x in x_vars)
    @time regmodel = lm(F, df)
    push!(result, regmodel)
end
result

  1.759678 seconds (6.59 M allocations: 447.620 MiB, 10.94% gc time)
  0.189806 seconds (223.53 k allocations: 14.381 MiB, 34.16% gc time, 2.09% compilation time)
  0.131929 seconds (223.53 k allocations: 14.366 MiB, 3.11% compilation time)
  0.125865 seconds (223.53 k allocations: 14.366 MiB, 3.21% compilation time)
  0.131615 seconds (223.54 k allocations: 14.377 MiB, 5.56% gc time, 3.07% compilation time)
  0.122072 seconds (223.54 k allocations: 14.368 MiB, 3.15% compilation time)
  0.124061 seconds (223.54 k allocations: 14.372 MiB, 3.22% compilation time)
  0.132258 seconds (223.53 k allocations: 14.369 MiB, 5.46% gc time, 3.22% compilation time)
  0.122774 seconds (223.54 k allocations: 14.368 MiB, 3.23% compilation time)
  0.121993 seconds (223.53 k allocations: 14.368 MiB, 3.15% compilation time)


10-element Vector{Any}:
 StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x7 + x11 + x5 + x10 + x16 + x18 + x20 + x9 + x4 + x6

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.934278    0.152924    6.11    <1e-07   0.630421    1.23813
x7            0.149379    0.106791    1.40    0.1653  -0.0628127   0.361571
x11          -0.150149    0.0999836  -1.50    0.1367  -0.348814    0.0485163
x5           -0.25046     0.104914   -2.39    0.0191  -0.458922   -0.0419989
x10          -0.0958473   0.101973   -0.94    0.3498  -0.298465    0.10677
x16          -0.165932    0.0973373  -1.70    0.0917  -0.359339    0.0274749
x18          -0.0341451   0.10571

## Solution

In [5]:
using DataFrames
using GLM
using StatsBase: sample

struct MyLinearModel{T, Y, X}
    linmodel::T
    y_var::Y
    x_vars::X
end

function my_lm(y_var, x_vars, df)
    y = df[!, y_var]
    X = [ones(nrow(df)) Matrix(df[!, x_vars])]
    linmodel = lm(X, y)
    MyLinearModel(linmodel, y_var, x_vars)
end

function Base.show(io::IO, mylm::MyLinearModel)
    linmodel, y_var, x_vars = getfield.(Ref(mylm), (:linmodel, :y_var, :x_vars))
    ct = coeftable(linmodel)
    ct.rownms .= string.((Symbol("(Intercept)"), x_vars...,))
    print(io, "Formula: ")
    print(io, y_var, " ~ 1")
    for x in x_vars print(io, " + ", x) end
    print(io, "\n\n")
    show(io, ct)
    print(io, "\n")
end

n = 20
r = 10
x_symbols = [Symbol("x$i") for i in 1:n]
df = DataFrame(rand(100, n+1), [:y; x_symbols]);

myresult = []
for _ in 1:10
    x_vars = sample(x_symbols, r; replace=false)
    @time mylinmodel = my_lm(:y, x_vars, df)
    push!(myresult, mylinmodel)
end
myresult

  0.255168 seconds (604.40 k allocations: 37.423 MiB, 4.24% gc time)
  0.000039 seconds (56 allocations: 43.516 KiB)
  0.000025 seconds (56 allocations: 43.516 KiB)
  0.000024 seconds (56 allocations: 43.516 KiB)
  0.000021 seconds (56 allocations: 43.516 KiB)
  0.000026 seconds (56 allocations: 43.516 KiB)
  0.000024 seconds (56 allocations: 43.516 KiB)
  0.000027 seconds (56 allocations: 43.516 KiB)
  0.000051 seconds (56 allocations: 43.516 KiB)
  0.000029 seconds (56 allocations: 43.516 KiB)


10-element Vector{Any}:
 Formula: y ~ 1 + x5 + x14 + x9 + x8 + x3 + x11 + x15 + x13 + x1 + x16

───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   0.457518     0.1633     2.80    0.0062   0.133043    0.781992
x5           -0.0290063    0.110364  -0.26    0.7933  -0.248298    0.190285
x14          -0.0775426    0.105422  -0.74    0.4639  -0.287015    0.131929
x9            0.0232791    0.109471   0.21    0.8321  -0.194237    0.240796
x8            0.063031     0.106069   0.59    0.5539  -0.147725    0.273787
x3            0.0448091    0.104059   0.43    0.6678  -0.161955    0.251573
x11          -0.103891     0.108845  -0.95    0.3424  -0.320163    0.112382
x15           0.0729766    0.109596   0.67    0.5072  -0.144789    0.290742
x13          -0.0153932    0.109127  -0.14    0.8881  -0.232226    0

## Analysis of the slow-down

In [6]:
x_vars = sample(x_symbols, r; replace=false)
@show x_vars
@time F = term(:y) ~ sum(term(x) for x in x_vars)
@time lm(F, df)

x_vars = [:x14, :x12, :x2, :x10, :x17, :x18, :x20, :x4, :x19, :x1]
  0.062505 seconds (79.68 k allocations: 5.127 MiB, 99.50% compilation time)
  0.149761 seconds (223.52 k allocations: 14.367 MiB, 6.96% gc time, 2.67% compilation time)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x14 + x12 + x2 + x10 + x17 + x18 + x20 + x4 + x19 + x1

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   0.483408    0.166988    2.89    0.0048   0.151607   0.815209
x14          -0.0892524   0.100795   -0.89    0.3783  -0.28953    0.111025
x12          -0.0547324   0.0992088  -0.55    0.5825  -0.251858   0.142393
x2            0.0612057   0.10779     0.57    0.5716  -0.152971   0.275383
x10          -0.095402    0.106615   -0.89    0.3733  -0.307243   0.116439
x17           0.0243093   0.10935     0.22    0.8246  -0.192968   0.241586
x18           0.221916    0.10478     2.12    0.0370   0.0137202  

In [7]:
@time my_lm(:y, x_vars, df)

  0.000212 seconds (56 allocations: 43.516 KiB)


Formula: y ~ 1 + x14 + x12 + x2 + x10 + x17 + x18 + x20 + x4 + x19 + x1

───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   0.483408    0.166988    2.89    0.0048   0.151607   0.815209
x14          -0.0892524   0.100795   -0.89    0.3783  -0.28953    0.111025
x12          -0.0547324   0.0992088  -0.55    0.5825  -0.251858   0.142393
x2            0.0612057   0.10779     0.57    0.5716  -0.152971   0.275383
x10          -0.095402    0.106615   -0.89    0.3733  -0.307243   0.116439
x17           0.0243093   0.10935     0.22    0.8246  -0.192968   0.241586
x18           0.221916    0.10478     2.12    0.0370   0.0137202  0.430111
x20          -0.0156434   0.102765   -0.15    0.8794  -0.219835   0.188548
x4            0.139605    0.103618    1.35    0.1813  -0.0662821  0.345492
x19          -0.0790359 

In [8]:
x_vars = sample(x_symbols, r; replace=false)
@time my_lm(:y, x_vars, df)

  0.000061 seconds (56 allocations: 43.516 KiB)


Formula: y ~ 1 + x11 + x18 + x19 + x13 + x20 + x8 + x6 + x12 + x17 + x9

─────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.444577      0.145853   3.05    0.0030   0.15477      0.734383
x11          -0.104502      0.104959  -1.00    0.3221  -0.313052     0.104049
x18           0.210382      0.101863   2.07    0.0418   0.00798149   0.412782
x19          -0.0930582     0.103526  -0.90    0.3711  -0.298762     0.112645
x13          -0.0422518     0.104517  -0.40    0.6870  -0.249924     0.16542
x20           0.00210056    0.103541   0.02    0.9839  -0.203634     0.207835
x8            0.0796087     0.103186   0.77    0.4425  -0.125421     0.284638
x6           -0.0649827     0.103492  -0.63    0.5317  -0.27062      0.140655
x12          -0.0306964     0.100007  -0.31    0.7596  -0.229408     0

In [9]:
x_vars = sample(x_symbols, r; replace=false)
@time F = term(:y) ~ sum(term(x) for x in x_vars)
@time cols = Tables.columntable(df)
@time mf = ModelFrame(F, cols, model=LinearModel)
@time mm = ModelMatrix(mf)
@time y = response(mf)
@time linmodel = fit(LinearModel, mm.m, y)
@time regmodel = StatsModels.TableRegressionModel(linmodel, mf, mm);

  0.076562 seconds (79.67 k allocations: 5.126 MiB, 99.52% compilation time)
  0.000022 seconds (29 allocations: 1.766 KiB)
  0.055888 seconds (136.64 k allocations: 8.861 MiB, 6.97% compilation time)
  0.064569 seconds (80.32 k allocations: 5.020 MiB, 99.56% compilation time)
  0.007192 seconds (3.77 k allocations: 282.191 KiB, 99.62% compilation time)
  0.000043 seconds (21 allocations: 23.188 KiB)
  0.003980 seconds (2.75 k allocations: 196.098 KiB, 94.19% compilation time)


In [10]:
typeof(cols)

NamedTuple{(:y, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10, :x11, :x12, :x13, :x14, :x15, :x16, :x17, :x18, :x19, :x20), NTuple{21, Vector{Float64}}}

In [11]:
typeof(mf)

ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}, LinearModel}

In [12]:
typeof(mm) |> x -> (fieldnames(x), fieldtypes(x))

((:m, :assign), (Matrix{Float64}, Vector{Int64}))

In [13]:
typeof(y)

Vector{Float64} (alias for Array{Float64, 1})

In [14]:
typeof(linmodel)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}

In [15]:
typeof(regmodel)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

In [16]:
regmodel.model == linmodel

true

In [17]:
regmodel.mf == mf

true

In [18]:
regmodel.mm == mm

true

In [19]:
@which lm(F, df)

lm(X, y) in GLM at D:\.julia\packages\GLM\5CcRd\src\lm.jl:179

In [20]:
@which fit(LinearModel, F, df)

fit(::Type{T}, f::FormulaTerm, data, args...; contrasts, kwargs...) where T<:RegressionModel in StatsModels at D:\.julia\packages\StatsModels\JZLpf\src\statsmodel.jl:78

In [21]:
@which fit(LinearModel, mm.m, y)

fit(::Type{LinearModel}, X::AbstractMatrix{<:Real}, y::AbstractVector{<:Real}) in GLM at D:\.julia\packages\GLM\5CcRd\src\lm.jl:156

In [22]:
@code_warntype term(:y) ~ sum(term(x) for x in x_vars)

MethodInstance for ~(::Term, ::NTuple{10, Term})
  from ~(lhs::Union{Tuple{AbstractTerm, Vararg{AbstractTerm}}, AbstractTerm}, rhs::Union{Tuple{AbstractTerm, Vararg{AbstractTerm}}, AbstractTerm}) in StatsModels at D:\.julia\packages\StatsModels\JZLpf\src\terms.jl:401
Arguments
  #self#::Core.Const(~)
  lhs::Term
  rhs::NTuple{10, Term}
Body::FormulaTerm{Term, NTuple{10, Term}}
1 ─ %1 = StatsModels.FormulaTerm(lhs, rhs)::FormulaTerm{Term, NTuple{10, Term}}
└──      return %1



In [23]:
@code_warntype Tables.columntable(df)

MethodInstance for Tables.columntable(::DataFrame)
  from columntable(itr::T) where T in Tables at D:\.julia\packages\Tables\gg6Id\src\namedtuples.jl:170
Static Parameters
  T = DataFrame
Arguments
  #self#::Core.Const(Tables.columntable)
  itr::DataFrame
Locals
  cols::DataFrames.DataFrameColumns{DataFrame}
Body::NamedTuple
1 ─      (cols = Tables.columns(itr))
│   %2 = (cols isa Tables.ColumnTable)::Core.Const(false)
└──      goto #3 if not %2
2 ─      Core.Const(:(return cols))
3 ┄ %5 = Tables.schema(cols)::Tables.Schema
│   %6 = Tables.columntable(%5, cols)::NamedTuple
└──      return %6



In [24]:
@code_warntype ModelFrame(F, cols, model=LinearModel)

MethodInstance for (::Core.var"#Type##kw")(::NamedTuple{(:model,), Tuple{UnionAll}}, ::Type{ModelFrame}, ::FormulaTerm{Term, NTuple{10, Term}}, ::NamedTuple{(:y, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10, :x11, :x12, :x13, :x14, :x15, :x16, :x17, :x18, :x19, :x20), NTuple{21, Vector{Float64}}})
  from (::Core.var"#Type##kw")(::Any, ::Type{ModelFrame}, f::FormulaTerm, data::NamedTuple{names, T} where {N, D, names, T<:Tuple{Vararg{AbstractArray{S, D} where S, N}}}) in StatsModels at D:\.julia\packages\StatsModels\JZLpf\src\modelframe.jl:72
Arguments
  _::Core.Const(Core.var"#Type##kw"())
  @_2::NamedTuple{(:model,), Tuple{UnionAll}}
  @_3::Type{ModelFrame}
  f::FormulaTerm{Term, NTuple{10, Term}}
  data::NamedTuple{(:y, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10, :x11, :x12, :x13, :x14, :x15, :x16, :x17, :x18, :x19, :x20), NTuple{21, Vector{Float64}}}
Locals
  model::UnionAll
  contrasts::Dict{Symbol, Any}
  @_8::UnionAll
  @_9::Dict{Symbol, Any}
Body::ModelFrame
1 ─ %1  =

In [25]:
@code_warntype ModelMatrix(mf)

MethodInstance for ModelMatrix(::ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}, LinearModel})
  from ModelMatrix(mf::ModelFrame) in StatsModels at D:\.julia\packages\StatsModels\JZLpf\src\modelframe.jl:222
Arguments
  #self#::Type{ModelMatrix}
  mf::ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}, LinearModel}
Body::ModelMatrix
1 ─ %1 = Core.apply_type(StatsModels.Matrix, StatsModels.Float64)::Core.Const(Matrix{Float64})
│   %2 = Core.apply_type(StatsModels.ModelMatrix, %1)::Core.Const(ModelMatrix{Matrix{Float64}})
│   %3 = (%2)(mf)::ModelMatrix
└──      return %3



In [26]:
@code_warntype response(mf)

MethodInstance for StatsBase.response(::ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}, LinearModel})
  from response(mf::ModelFrame; data) in StatsModels at D:\.julia\packages\StatsModels\JZLpf\src\modelframe.jl:148
Arguments
  #self#::Core.Const(StatsBase.response)
  mf::ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}, LinearModel}
Locals
  data::NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}
Body::Any
1 ─ %1 = Base.getproperty(mf, :data)::NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}
│        (data = %1)
│   %3 = StatsModels.:(var"#response#64")(data, #self#, mf)::Any
└──      return %3



In [27]:
@code_warntype fit(LinearModel, mm.m, y)

MethodInstance for StatsBase.fit(::Type{LinearModel}, ::Matrix{Float64}, ::Vector{Float64})
  from fit(::Type{LinearModel}, X::AbstractMatrix{<:Real}, y::AbstractVector{<:Real}) in GLM at D:\.julia\packages\GLM\5CcRd\src\lm.jl:156
Arguments
  #self#::Core.Const(StatsBase.fit)
  @_2::Type{LinearModel}
  X::Matrix{Float64}
  y::Vector{Float64}
Body::Union{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}}
1 ─ %1 = (#self#)(@_2, X, y, GLM.nothing)::Union{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}}
└──      return %1



In [28]:
@code_warntype StatsModels.TableRegressionModel(linmodel, mf, mm)

MethodInstance for StatsModels.TableRegressionModel(::LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, ::ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float64}}}, LinearModel}, ::ModelMatrix{Matrix{Float64}})
  from StatsModels.TableRegressionModel(model::M, mf::ModelFrame, mm::ModelMatrix{T}) where {M, T} in StatsModels at D:\.julia\packages\StatsModels\JZLpf\src\statsmodel.jl:70
Static Parameters
  M = LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}
  T = Matrix{Float64}
Arguments
  #self#::Type{StatsModels.TableRegressionModel}
  model::LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}
  mf::ModelFrame{NamedTuple{(:y, :x15, :x17, :x10, :x5, :x12, :x13, :x7, :x20, :x18, :x3), NTuple{11, Vector{Float6

In [29]:
@code_warntype my_lm(:y, x_vars, df)

MethodInstance for my_lm(::Symbol, ::Vector{Symbol}, ::DataFrame)
  from my_lm(y_var, x_vars, df) in Main at In[5]:11
Arguments
  #self#::Core.Const(my_lm)
  y_var::Symbol
  x_vars::Vector{Symbol}
  df::DataFrame
Locals
  linmodel::LinearModel
  X::Matrix
  y::AbstractVector
Body::MyLinearModel{_A, Symbol, Vector{Symbol}} where _A
1 ─      (y = Base.getindex(df, Main.:!, y_var))
│   %2 = Main.nrow(df)::Int64
│   %3 = Main.ones(%2)::Vector{Float64}
│   %4 = Base.getindex(df, Main.:!, x_vars)::DataFrame
│   %5 = Main.Matrix(%4)::Matrix
│        (X = Base.hcat(%3, %5))
│        (linmodel = Main.lm(X, y))
│   %8 = Main.MyLinearModel(linmodel, y_var, x_vars)::MyLinearModel{_A, Symbol, Vector{Symbol}} where _A
└──      return %8

